# ============================================
# CELL 1: Imports and Setup
# ============================================

In [60]:
from google.colab import drive
drive.mount('/content/drive')

# Install packages
!pip install faker imbalanced-learn xgboost shap optuna -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
# Clone your repo in Colab
!git clone https://github.com/YOUR_USERNAME/business-analytics-ml.git
%cd business-analytics-ml

# Configure git
!git config --global user.email "asthapankaj2503@gmail.com"
!git config --global user.name "Asthaasu"

Cloning into 'business-analytics-ml'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'business-analytics-ml'
/content/business-analytics-ml/business-analytics-ml


In [62]:
# ============================================
# CELL 1: Load Data
# ============================================
import pandas as pd
import numpy as np
from datetime import datetime

customers = pd.read_csv('data/raw/customers.csv', parse_dates=['signup_date'])
transactions = pd.read_csv('data/raw/transactions.csv', parse_dates=['transaction_date'])
support_tickets = pd.read_csv('data/raw/support_tickets.csv', parse_dates=['ticket_date'])

print("✓ Data loaded")

# ============================================
# CELL 2: RFM Features (Most Important!)
# ============================================
reference_date = pd.Timestamp('2024-12-31')

print("Creating RFM features...")

rfm = transactions.groupby('customer_id').agg({
    'transaction_date': lambda x: (reference_date - x.max()).days,  # Recency
    'transaction_id': 'count',  # Frequency
    'amount': ['sum', 'mean', 'std', 'min', 'max']  # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'recency_days', 'transaction_count',
               'total_spend', 'avg_transaction_value', 'std_transaction_value',
               'min_transaction', 'max_transaction']

rfm['std_transaction_value'] = rfm['std_transaction_value'].fillna(0)

print(f"✓ RFM features created for {len(rfm)} customers")

# ============================================
# CELL 3: Engagement & Behavioral Features
# ============================================
print("Creating engagement features...")

# Time-weighted engagement (recent activity weighted more)
transactions['days_ago'] = (reference_date - transactions['transaction_date']).dt.days
transactions['decay_weight'] = np.exp(-transactions['days_ago'] / 90)  # 90-day half-life

engagement = transactions.groupby('customer_id').agg({
    'decay_weight': 'sum',
    'product_category': 'nunique'
}).reset_index()

engagement.columns = ['customer_id', 'engagement_score', 'product_diversity']

# Trend analysis: comparing recent vs. old spending
transactions_sorted = transactions.sort_values(['customer_id', 'transaction_date'])

# Last 30 days vs. previous 60 days
transactions['is_last_30'] = transactions['days_ago'] <= 30
transactions['is_prev_60'] = (transactions['days_ago'] > 30) & (transactions['days_ago'] <= 90)

spending_trend = transactions.groupby('customer_id').apply(
    lambda x: pd.Series({
        'spend_last_30': x[x['is_last_30']]['amount'].sum(),
        'spend_prev_60': x[x['is_prev_60']]['amount'].sum(),
    })
).reset_index()

spending_trend['spending_trend'] = (
    (spending_trend['spend_last_30'] - spending_trend['spend_prev_60']) /
    (spending_trend['spend_prev_60'] + 1)  # Avoid division by zero
)

print("✓ Engagement features created")

# ============================================
# CELL 4: Support Ticket Features
# ============================================
print("Creating support ticket features...")

ticket_features = support_tickets.groupby('customer_id').agg({
    'ticket_id': 'count',
    'priority': lambda x: (x == 'High').sum(),
    'resolved': lambda x: x.mean()
}).reset_index()

ticket_features.columns = ['customer_id', 'ticket_count', 'high_priority_tickets', 'resolution_rate']

print("✓ Support ticket features created")

# ============================================
# CELL 5: Customer Tenure & Contract Features
# ============================================
print("Creating customer-level features...")

customers['days_since_signup'] = (reference_date - pd.to_datetime(customers['signup_date'])).dt.days

# One-hot encode categorical variables
customers['contract_is_monthly'] = (customers['contract_type'] == 'monthly').astype(int)
customers['contract_is_annual'] = (customers['contract_type'] == 'annual').astype(int)
customers['has_account_manager'] = customers['account_manager_assigned'].astype(int)
customers['is_enterprise'] = (customers['company_size'] == 'Enterprise').astype(int)

print("✓ Customer features created")

# ============================================
# CELL 6: Merge All Features
# ============================================
print("Merging all features...")

# Start with customers
features = customers[['customer_id', 'days_since_signup', 'contract_is_monthly',
                      'contract_is_annual', 'has_account_manager', 'is_enterprise',
                      'age', 'churned']]

# Merge RFM
features = features.merge(rfm, on='customer_id', how='left')

# Merge engagement
features = features.merge(engagement, on='customer_id', how='left')

# Merge spending trend
features = features.merge(spending_trend[['customer_id', 'spending_trend']],
                          on='customer_id', how='left')

# Merge ticket features
features = features.merge(ticket_features, on='customer_id', how='left')

# Fill missing values (customers with no transactions/tickets)
features = features.fillna(0)

print(f"✓ Final feature set: {features.shape}")
print(f"  Features: {features.shape[1] - 2}")  # Exclude customer_id and churned

# ============================================
# CELL 7: Feature Statistics
# ============================================
print("\n=== FEATURE STATISTICS ===")
print("\nTop 10 features by variance:")
numeric_features = features.select_dtypes(include=[np.number]).columns
feature_variance = features[numeric_features].var().sort_values(ascending=False)
print(feature_variance.head(10))

print("\nFeature correlations with churn:")
correlations = features[numeric_features].corrwith(features['churned']).sort_values(ascending=False)
print(correlations)

# ============================================
# CELL 8: Save Processed Features
# ============================================
features.to_csv('data/processed/features_churn.csv', index=False)
print("\n✓ Features saved to data/processed/features_churn.csv")

# Print summary
print(f"\n=== FINAL DATASET ===")
print(f"Total customers: {len(features)}")
print(f"Total features: {len(features.columns) - 2}")
print(f"Churned: {features['churned'].sum()} ({features['churned'].mean():.1%})")
print(f"Active: {(features['churned'] == 0).sum()} ({(features['churned'] == 0).mean():.1%})")

✓ Data loaded
Creating RFM features...
✓ RFM features created for 10000 customers
Creating engagement features...
✓ Engagement features created
Creating support ticket features...
✓ Support ticket features created
Creating customer-level features...
✓ Customer features created
Merging all features...
✓ Final feature set: (10000, 21)
  Features: 19

=== FEATURE STATISTICS ===

Top 10 features by variance:
total_spend              1.485610e+10
customer_id              8.334167e+06
max_transaction          1.127645e+06
avg_transaction_value    6.049676e+05
min_transaction          2.467453e+05
days_since_signup        2.260043e+05
recency_days             1.015403e+05
std_transaction_value    2.848979e+04
transaction_count        2.558150e+03
age                      2.776145e+02
dtype: float64

Feature correlations with churn:
churned                  1.000000
recency_days             0.867582
spending_trend           0.777413
contract_is_monthly      0.186886
ticket_count             0.